In [6]:
import numpy as np
import glob
import pandas as pd

In [29]:
#step 1 - combining all daily files into one file
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [30]:
#checking to see if the files all combined
pd.read_csv("combined_csv.csv")

C:\Users\David Jhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ticker,date,url,num_mentions,pct_mentions,pos_count,neg_count,bullish_pct,bearish_pct,neutral_pct,price,price_change_net,price_change_pct,time_of_price
0,GME,"Dec 28, 2020",https://finance.yahoo.com/quote/GME?p=GME,567,20,257,131,45,23,31,20.99,0.84,+4.17%,At close: 4:00PM EST
1,PLTR,"Dec 28, 2020",https://finance.yahoo.com/quote/PLTR?p=PLTR,418,14,164,110,39,26,34,25.63,-2.12,-7.64%,At close: 4:00PM EST
2,PSTH,"Dec 28, 2020",https://finance.yahoo.com/quote/PSTH?p=PSTH,80,2,51,19,63,23,12,25.99,-0.04,-0.15%,At close: 4:00PM EST
3,PSA,"Dec 28, 2020",https://finance.yahoo.com/quote/PSA?p=PSA,3,<1,0,0,0,0,100,229.14,0.29,+0.13%,At close: 4:00PM EST
4,TD,"Dec 28, 2020",https://finance.yahoo.com/quote/TD?p=TD,9,<1,2,1,22,11,66,56.04,-0.04,-0.07%,At close: 4:00PM EST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162843,NNDM,"Oct 08, 2021",https://finance.yahoo.com/quote/NNDM?p=NNDM,1,<1,0,0,0,0,100,5.68,0.16,+2.90%,At close: 4:00PM EDT
162844,SP,"Oct 08, 2021",https://finance.yahoo.com/quote/SP?p=SP,1,<1,1,0,100,0,0,NaN,NaN,NaN,NaN
162845,COIN,"Oct 08, 2021",https://finance.yahoo.com/quote/COIN?p=COIN,3,<1,3,0,100,0,0,251.59,1.21,+0.48%,At close: 4:00PM EDT
162846,ZM,"Oct 08, 2021",https://finance.yahoo.com/quote/ZM?p=ZM,1,<1,1,0,100,0,0,256.27,2.62,+1.03%,At close: 4:00PM EDT


In [31]:
# setting df as a DataFrame as combined_csv
#drop blanks from the ticker
df = pd.read_csv("combined_csv.csv")
df.dropna(subset=['ticker'], inplace=True)
df

C:\Users\David Jhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ticker,date,url,num_mentions,pct_mentions,pos_count,neg_count,bullish_pct,bearish_pct,neutral_pct,price,price_change_net,price_change_pct,time_of_price
0,GME,"Dec 28, 2020",https://finance.yahoo.com/quote/GME?p=GME,567,20,257,131,45,23,31,20.99,0.84,+4.17%,At close: 4:00PM EST
1,PLTR,"Dec 28, 2020",https://finance.yahoo.com/quote/PLTR?p=PLTR,418,14,164,110,39,26,34,25.63,-2.12,-7.64%,At close: 4:00PM EST
2,PSTH,"Dec 28, 2020",https://finance.yahoo.com/quote/PSTH?p=PSTH,80,2,51,19,63,23,12,25.99,-0.04,-0.15%,At close: 4:00PM EST
3,PSA,"Dec 28, 2020",https://finance.yahoo.com/quote/PSA?p=PSA,3,<1,0,0,0,0,100,229.14,0.29,+0.13%,At close: 4:00PM EST
4,TD,"Dec 28, 2020",https://finance.yahoo.com/quote/TD?p=TD,9,<1,2,1,22,11,66,56.04,-0.04,-0.07%,At close: 4:00PM EST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162843,NNDM,"Oct 08, 2021",https://finance.yahoo.com/quote/NNDM?p=NNDM,1,<1,0,0,0,0,100,5.68,0.16,+2.90%,At close: 4:00PM EDT
162844,SP,"Oct 08, 2021",https://finance.yahoo.com/quote/SP?p=SP,1,<1,1,0,100,0,0,NaN,NaN,NaN,NaN
162845,COIN,"Oct 08, 2021",https://finance.yahoo.com/quote/COIN?p=COIN,3,<1,3,0,100,0,0,251.59,1.21,+0.48%,At close: 4:00PM EDT
162846,ZM,"Oct 08, 2021",https://finance.yahoo.com/quote/ZM?p=ZM,1,<1,1,0,100,0,0,256.27,2.62,+1.03%,At close: 4:00PM EDT


In [32]:
#drop TRUE from ticker
df['ticker'].replace('TRUE', np.nan, inplace=True)
df.dropna(subset=['ticker'], inplace=True)
df

,ticker,date,url,num_mentions,pct_mentions,pos_count,neg_count,bullish_pct,bearish_pct,neutral_pct,price,price_change_net,price_change_pct,time_of_price
0,GME,"Dec 28, 2020",https://finance.yahoo.com/quote/GME?p=GME,567,20,257,131,45,23,31,20.99,0.84,+4.17%,At close: 4:00PM EST
1,PLTR,"Dec 28, 2020",https://finance.yahoo.com/quote/PLTR?p=PLTR,418,14,164,110,39,26,34,25.63,-2.12,-7.64%,At close: 4:00PM EST
2,PSTH,"Dec 28, 2020",https://finance.yahoo.com/quote/PSTH?p=PSTH,80,2,51,19,63,23,12,25.99,-0.04,-0.15%,At close: 4:00PM EST
3,PSA,"Dec 28, 2020",https://finance.yahoo.com/quote/PSA?p=PSA,3,<1,0,0,0,0,100,229.14,0.29,+0.13%,At close: 4:00PM EST
4,TD,"Dec 28, 2020",https://finance.yahoo.com/quote/TD?p=TD,9,<1,2,1,22,11,66,56.04,-0.04,-0.07%,At close: 4:00PM EST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162843,NNDM,"Oct 08, 2021",https://finance.yahoo.com/quote/NNDM?p=NNDM,1,<1,0,0,0,0,100,5.68,0.16,+2.90%,At close: 4:00PM EDT
162844,SP,"Oct 08, 2021",https://finance.yahoo.com/quote/SP?p=SP,1,<1,1,0,100,0,0,NaN,NaN,NaN,NaN
162845,COIN,"Oct 08, 2021",https://finance.yahoo.com/quote/COIN?p=COIN,3,<1,3,0,100,0,0,251.59,1.21,+0.48%,At close: 4:00PM EDT
162846,ZM,"Oct 08, 2021",https://finance.yahoo.com/quote/ZM?p=ZM,1,<1,1,0,100,0,0,256.27,2.62,+1.03%,At close: 4:00PM EDT


In [ ]:
#No ticker should be more than 6, need to remove
ticker_6 = df['ticker'].str.len() > 6
df['ticker'].replace(ticker_6, np.nan, inplace=True)
df.dropna(subset=['ticker'], inplace=True)
df